In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
df = pd.read_csv("Dataset .csv", encoding='latin1')
df.head()

,ï»¿Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [8]:
df = df[['Restaurant Name', 'City', 'Locality', 'Cuisines',
         'Average Cost for two', 'Aggregate rating', 'Votes']]
df.dropna(inplace=True)

In [9]:
df['Cuisines'] = df['Cuisines'].astype(str)
df['City'] = df['City'].astype(str)
df['Locality'] = df['Locality'].astype(str)
df['Average Cost for two'] = df['Average Cost for two'].astype(int)

In [10]:
df['content'] = df['Cuisines'] + " " + df['City'] + " " + df['Locality']

In [11]:
vectorizer = TfidfVectorizer(stop_words='english')
tf_matrix = vectorizer.fit_transform(df['content'])

In [12]:
similarity_matrix = cosine_similarity(tf_matrix)

In [13]:
def recommend_restaurants(cuisine, city, max_budget=500, top_n=5):

    user_input = cuisine + " " + city
    user_vec = vectorizer.transform([user_input])

    similarity_score = cosine_similarity(user_vec, tf_matrix).flatten()

    df['similarity'] = similarity_score

    filtered = df[
        (df['City'].str.lower() == city.lower()) &
        (df['Average Cost for two'] <= max_budget)
    ]

    if filtered.empty:
        return "No restaurants found for the given preferences."

    filtered = filtered.sort_values(
        by=['similarity', 'Aggregate rating', 'Votes'],
        ascending=False
    )

    return filtered[['Restaurant Name', 'City', 'Locality',
                      'Cuisines', 'Average Cost for two',
                      'Aggregate rating', 'Votes']].head(top_n)


In [14]:
op = recommend_restaurants("North Indian", "Bangalore", 600)
print(op)

    Restaurant Name       City               Locality  \
732          Onesta  Bangalore            Indiranagar   
733          Onesta  Bangalore               JP Nagar   
741          Onesta  Bangalore           New BEL Road   
737      Eat Street  Bangalore  Koramangala 6th Block   

                                              Cuisines  Average Cost for two  \
732                               Pizza, Cafe, Italian                   600   
733                               Pizza, Cafe, Italian                   600   
741                               Pizza, Cafe, Italian                   600   
737  North Indian, Chinese, Italian, Street Food, D...                   400   

     Aggregate rating  Votes  
732               4.3   1413  
733               4.6    781  
741               4.6    627  
737               4.3    753  
